In [1]:
import datetime as dt

import panel as pn
import pandas as pd
import holoviews as hv
import hvplot.pandas
hvplot.extension("bokeh")

pn.config.theme = "dark"


def get_series(**options):
    df = pd.read_csv(options["url"], index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(options["tz"])
    df = df.sort_index(ascending=True)
    return df[[options["column"]]].rename(columns={options["column"]: options["name"]})


def get_dataframe(**options):
    df = pd.read_csv(options["url"], index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(options["tz"])
    df = df.sort_index(ascending=True)
    return df


active_tools = []
backend_opts={"plot.toolbar.autohide": True}
height=350
width=750

start_time = dt.datetime.now(dt.UTC) - dt.timedelta(days=31)
start_time

datetime.datetime(2024, 4, 28, 1, 34, 12, 87278, tzinfo=datetime.timezone.utc)

In [2]:
options = dict(
    url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/gridwatch.ca/hourly/summary.csv",
    tz="America/Toronto",
    column="",
)

df = get_dataframe(**options)
df = df[df.index > start_time]
df.head()

,Imports (MW),Exports (MW),Net Import/Exports (MW),Power Generated (MW),Ontario Demand (MW),Total Emissions (tonnes),CO2e Intensity (g/kWh),nuclear (%),hydro (%),gas (%),wind (%),biofuel (%),solar (%),nuclear (MW),hydro (MW),gas (MW),wind (MW),biofuel (MW),solar (MW)
2024-04-27 22:00:00-04:00,9.0,1628.0,1619.0,15702.0,14083.0,263.0,17.0,36.6,30.3,5.6,27.5,0.0,0.0,5750.0,4756.0,885.0,4311.0,0.0,0.0
2024-04-27 23:00:00-04:00,9.0,1956.0,1947.0,15345.0,13398.0,263.0,17.0,37.5,28.8,5.8,27.9,0.0,0.0,5750.0,4423.0,884.0,4288.0,0.0,0.0
2024-04-28 00:00:00-04:00,9.0,2522.0,2513.0,15116.0,12603.0,263.0,17.0,37.9,29.4,4.2,28.5,0.0,0.0,5736.0,4439.0,638.0,4302.0,1.0,0.0
2024-04-28 01:00:00-04:00,35.0,1982.0,1947.0,13884.0,11937.0,131.0,9.0,41.3,29.4,1.7,27.6,0.1,0.0,5735.0,4075.0,237.0,3828.0,9.0,0.0
2024-04-28 02:00:00-04:00,35.0,2186.0,2151.0,13549.0,11398.0,102.0,8.0,42.3,29.0,1.2,27.4,0.1,0.0,5732.0,3934.0,163.0,3709.0,11.0,0.0


In [3]:
plot_options = dict(
    value_label="MW",
    legend="bottom",
    title="Ontario grid supply and demand",
    height=height,
    width=width,
    stacked=False,    
    grid=True,
    # ylim=(0, None),
    # alpha=0.5,
    # hover=False,
)
columns = {
    "Power Generated (MW)": "generated",
    "Ontario Demand (MW)": "demand",
    "Imports (MW)": "imports",
    "Exports (MW)": "exports",
    "Net Import/Exports (MW)": "net (exports-imports)",
}
suply_demand = df[columns.keys()].rename(columns=columns).hvplot.line(**plot_options).opts(active_tools=active_tools, backend_opts=backend_opts)
suply_demand

:NdOverlay   [Variable]
   :Curve   [index]   (MW)

In [4]:
plot_options = dict(
    value_label="MW",
    legend="bottom",
    title="Ontario grid generation",
    height=height,
    width=width,
    grid=True,
    stacked=True,
    ylim=(0, None),
    alpha=0.5,
    hover=False,
)

columns = {x: x.replace(" (MW)", "") for x in df.columns if x.endswith(" (MW)") and (x[0]==x[0].lower())}
generation = df[columns.keys()].rename(columns=columns).hvplot.area(**plot_options).opts(active_tools=active_tools, backend_opts=backend_opts)
generation

:NdOverlay   [Variable]
   :Area   [index]   (MW,Baseline)

In [5]:
plot_options = dict(
    value_label="%",
    legend="bottom",
    title="Ontario grid relative power mix (%)",
    height=height,
    width=width,
    grid=True,
    stacked=True,
    ylim=(0, 100),
    alpha=0.5,
    hover=False,
)

columns = {x: x.replace(" (%)", "") for x in df.columns if x.endswith(" (%)")}
generation_pct = df[columns.keys()].rename(columns=columns).hvplot.area(**plot_options).opts(active_tools=active_tools, backend_opts=backend_opts)
generation_pct

:NdOverlay   [Variable]
   :Area   [index]   (%,Baseline)

In [6]:
options_gridwatch = dict(
    name = "gridwatch",
    url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/gridwatch.ca/hourly/summary.csv",
    tz="America/Toronto",
    column = "CO2e Intensity (g/kWh)"
)

options_co2signal = dict(
    name="co2signal",
    url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/co2signal.com/CA-ON/hourly/output.csv",
    column = "data.carbonIntensity",
    tz="America/Toronto",
)

df = get_series(**options_gridwatch).join(
    get_series(**options_co2signal),
    how="inner"
)

plot_options = dict(
    value_label="g/kWh",
    legend="bottom",
    title="Ontario grid co2 emissions intensity",
    height=height,
    width=width,
    grid=True,
    ylim=(0, None),
    # alpha=0.5,
    # hover=False,
)
df = df[df.index > start_time]

range_select = pn.widgets.DatetimeRangePicker(value=(df.index[0].to_pydatetime(), df.index[-1].to_pydatetime()))
co2_intensity = df.hvplot.line(**plot_options).apply.opts(active_tools=active_tools, backend_opts=backend_opts, xlim=range_select, framewise=True)
rxy = hv.streams.RangeX(source=co2_intensity)

def update_widget(event):
    new_dates = tuple([pd.Timestamp(i).to_pydatetime() for i in event.new])
    if new_dates != range_select.value:
        range_select.value = new_dates
rxy.param.watch(update_widget, 'x_range')

co2_intensity

Traceback (most recent call last):
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\viewable.py", line 469, in _on_msg
 patch = manager.assemble(msg)
 ^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\models\comm_manager.py", line 29, in assemble
 msg_obj = cls(header, msg['metadata'], msg['content'])
 ~~~^^^^^^^^^^^
KeyError: 'content'
 


 Traceback (most recent call last):
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\viewable.py", line 469, in _on_msg
 patch = manager.assemble(msg)
 ^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\models\comm_manager.py", line 29, in assemble
 msg_obj = cls(header, msg['metadata'], msg['content'])
 ~~~^^^^^^^^^^^
KeyError: 'content'
 


 Traceback (most recent call last):
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\viewable.py", line 469, in _on_msg
 patch = manager.assemble(msg)
 ^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\models\comm_manager.py", line 29, in assemble
 msg_obj = cls(header, msg['metadata'], msg['content'])
 ~~~^^^^^^^^^^^
KeyError: 'content'
 


 Traceback (most recent call last):
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\viewable.py", line 469, in _on_msg
 patch = manager.assemble(msg)
 ^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\models\comm_manager.py", line 29, in assemble
 msg_obj = cls(header, msg['metadata'], msg['content'])
 ~~~^^^^^^^^^^^
KeyError: 'content'
 


 Traceback (most recent call last):
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\viewable.py", line 469, in _on_msg
 patch = manager.assemble(msg)
 ^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\models\comm_manager.py", line 29, in assemble
 msg_obj = cls(header, msg['metadata'], msg['content'])
 ~~~^^^^^^^^^^^
KeyError: 'content'
 


 Traceback (most recent call last):
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\viewable.py", line 469, in _on_msg
 patch = manager.assemble(msg)
 ^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\models\comm_manager.py", line 29, in assemble
 msg_obj = cls(header, msg['metadata'], msg['content'])
 ~~~^^^^^^^^^^^
KeyError: 'content'
 


 Traceback (most recent call last):
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\viewable.py", line 469, in _on_msg
 patch = manager.assemble(msg)
 ^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\models\comm_manager.py", line 29, in assemble
 msg_obj = cls(header, msg['metadata'], msg['content'])
 ~~~^^^^^^^^^^^
KeyError: 'content'
 


 Traceback (most recent call last):
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\ryan\miniconda3\envs\pn\Lib\site-packages\panel\viewable.py", line 469, in _on_msg
 patch = manager.assemble(msg)
 ^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\ryan\mi

:DynamicMap   []
   :NdOverlay   [Variable]
      :Curve   [index]   (g/kWh)

In [11]:
template = pn.template.FastGridTemplate(
    row_height=200,
    theme_toggle=False,
    theme="dark",
    title="Ontario grid data",
    prevent_collision=False,
    sidebar=[range_select],
    collapsed_sidebar=True,
)
template.main[0:2,0:6]=co2_intensity
template.main[0:2,6:12]=generation_pct
template.main[2:4,0:6]=suply_demand
template.main[2:4,6:12]=generation

In [8]:
template.servable();

In [13]:
#!panel convert index.ipynb --to pyodide-worker --out ..\docs --pwa --title "Ontario grid data"

Successfully converted index.ipynb to pyodide-worker target and wrote output to index.html.
Successfully wrote icons and images.
Successfully wrote site.manifest.
Successfully wrote serviceWorker.js.


http://localhost:8000/docs

In [10]:
#!python -m http.server